In [ ]:
import pandas as pd
import numpy as np
import math
import nltk
from nltk.corpus import stopwords
import re
import string
from collections import Counter
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
%matplotlib inline
import seaborn as sns

df1 = pd.read_csv('../book-reccomendation-engine/books_cleaned.csv')
df2 = pd.read_csv('../keyword_engine/books_authors_final.csv')
book_titles_dict1 = df1['book_title'].to_dict()
opt1 = []
for k,v in book_titles_dict1.items():
    temp_d = {}
    temp_d["label"] = v
    temp_d["value"] = v
    opt1.append(temp_d)
book_titles_dict2 = df2['title'].to_dict()
opt2 = []
for k,v in book_titles_dict2.items():
    temp_d = {}
    temp_d["label"] = v
    temp_d["value"] = v
    opt2.append(temp_d)
def book_engine(book):
    titles = list(df2['title'])
    response = requests.get(df2.iloc[0]['image_url'])
    img = Image.open(BytesIO(response.content))

    index = titles.index(book)
    authorsList = df2.iloc[index]['authors'].split("|")
    authorStr = ""
    for i in range(len(authorsList)):
        authorStr += authorsList[i]
        if i == len(authorsList) - 1:
            continue
        authorStr += " | "
    return [df2.iloc[index]['title'], authorStr,
            df2.iloc[index]['average rating'], df2.iloc[index]['genres'], df2.iloc[index]['number of pages'], df2.iloc[index]["reviews' keywords"], df2.iloc[index]['description'],
            df2.iloc[index]['more about author(s)'], df2.iloc[index]['image_url']]
def get_cosine_sim(*strs):
    #     print(strs)
    vectors = [t for t in get_vectors(*strs)]
    return cosine(*vectors)

def get_jaccard_sim(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def get_vectors(*strs):
    text = []
    for t in strs:
        t = t.translate(str.maketrans('', '', string.punctuation))
        stop_words = stopwords.words('english')
        stopwords_dict = Counter(stop_words)
        text.append(' '.join([word for word in t.split() if word not in stopwords_dict]))

    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()
def recommend_desc(book):

    if len(df1[df1['book_title'] == book.lower()]) > 0:
        desc = list(df1[df1['book_title'] == book.lower()]['book_desc'])[0]
        print('Found match: ', book, '\n')
        match = book
    else:
        index = np.argmax([fuzz.ratio(book.lower(), i) for i in list(df1['book_title']) if type(i)== str])
        desc = df1.iloc[index,:]['book_desc']
        print('Found closest match: ', df1.iloc[index,:]['book_title'], '\n')
        match = df1.iloc[index,:]['book_title']

    all_desc = list(df1['book_desc'])
    all_genres = list(df1['genres'])
    similarity_array = np.zeros([len(all_desc),])

    genre_input = df1.iloc[df1[df1['book_title'] == match].index[0],:]['genres'].split('|')
    
    for k, i in enumerate(all_desc):
        if len(list(set(genre_input) & set(all_genres[k].split('|')))) > 2:
            if type(i)==str:
                value = get_cosine_sim(i, desc)
                if value != math.nan:
                    similarity_array[k] = get_cosine_sim(desc, i)
    similarity = similarity_array.tolist()
    similarity.remove(max(similarity))
    final_index = np.nanargmax(similarity)

    response = requests.get(df1.iloc[final_index,:]['image_url'])

    img = Image.open(BytesIO(response.content))

    return [match, df1.iloc[final_index,:]['book_title'], df1.iloc[final_index,:]['book_desc'],
            df1.iloc[final_index,:]['book_rating'], df1.iloc[final_index,:]['book_pages'], df1.iloc[final_index,:]['book_authors'], df1.iloc[final_index,:]['image_url']]
app = dash.Dash()
mint = '#6dc9c0'
sky = '#8cd6e6'
tabs_styles = {
    'height': '60px'
}

tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'vertical-align': 'middle',
    'color':sky,
#     'padding': '0px',
    'fontSize': 20
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': 'white',
    'color': '#57d2eb',
    'fontWeight': 'bold',
    'fontSize': 20,
    'vertical-align': 'middle'
}

h6={"color": 'black',
    "font-size":'20px',
    'padding':'1px',
    "margin-top": "1px",
    "margin-top": "1px"
#     "margin-left": "10px"
   }
    
    
    

app.layout = html.Div([
    
    html.H1('Book Dashboard',
            style={'color':mint,
                   'margin':'auto',
                   'margin-top':'10px',
                   'margin-bottom':'10px',
                   'text-align': 'center'
                  }
           ),
    
    dcc.Tabs(id='tabs-example', 
             value='Data Exploration', 
             children=[
                dcc.Tab(label='Data Exploration based on Generes', 
                        value='Data Exploration based on Generes',
                        style=tab_style, selected_style=tab_selected_style),
                dcc.Tab(label='Search Engine', 
                        value='Search Engine',
                        style=tab_style, selected_style=tab_selected_style),
                dcc.Tab(label='Make a Recommandation', 
                        value='Make a Recommandation',
                        style=tab_style, selected_style=tab_selected_style)],
             style=tabs_styles),
    
    html.Div(id='tabs-example-content',style={'width':'80%','margin-top':'10px','margin':'auto'})
    ])



@app.callback(Output('tabs-example-content', 'children'),
              Input('tabs-example', 'value'))
def render_content(tab):
    
    if tab == 'Data Exploration based on Generes':
        return [
#             html.H3('Data Exploration based on Attributes',style={'color':'#1fccbb'}),
            dcc.Graph(
                id='graph_attributes'
            ),
            html.Div('Select an Attribute:', style={'text-align': 'center', 
                                                    'margin':'auto',
                                                    'margin-top':'10px',
                                                    'margin-bottom':'10px',
                                                    'color': mint, 
                                                    'fontSize': 25,
                                                    'font-weight': 'bold'}),
            dcc.Dropdown(
                id = 'column_seperated',
                options=[
                    {'label': 'Book rating', 'value': 'book_rating','color':'#57d2eb'},
                    {'label': 'Book rating count', 'value': 'book_rating_count'},
                    {'label': 'Book review count', 'value': 'book_review_count'},
                    {'label': 'Book pages', 'value': 'book_pages'},
                    {'label': 'Book format', 'value': 'book_format'},
                    {'label': 'Amazon recommanded', 'value': 'Amzn_rcmd'}
                            ],
                value='book_rating'
            ),
            html.Div('Select an Genere:', style={'text-align': 'center', 
                                                    'margin':'auto',
                                                    'margin-top':'10px',
                                                    'margin-bottom':'10px',
                                                    'color': mint, 
                                                    'fontSize': 25,
                                                    'font-weight': 'bold'}),
            dcc.Dropdown(
                id = 'generes',
                options=[
                    {'label': 'All', 'value': 'All'},
                    {'label': 'Young Adult', 'value': 'Young Adult'},
                    {'label': 'Fiction', 'value': 'Fiction'},
                    {'label': 'Science', 'value': 'Science'},
                    {'label': 'Dystopia', 'value': 'Dystopia'},
                    {'label': 'Fantasy', 'value': 'Fantasy'},
                    {'label': 'Classics', 'value': 'Classics'},
                    {'label': 'Historical', 'value': 'Historical'},
                    {'label': 'Academic', 'value': 'Academic'},
                    {'label': 'Romance', 'value': 'Romance'},
                    {'label': 'Paranormal', 'value': 'Paranormal'},
                    {'label': 'Literature', 'value': 'Literature'},
                    {'label': 'Adventure', 'value': 'Adventure'},
                    {'label': 'Mythology', 'value': 'Mythology'},
                    {'label': 'Childrens', 'value': 'Childrens'},
                    {'label': 'Horror', 'value': 'Horror'},
                    {'label': 'Psychology', 'value': 'Psychology'},
                    {'label': 'Philosophy', 'value': 'Philosophy'}
                            ],
                value='All'
            ),
            html.Div('Percent of Data to Display:', style={'text-align': 'center', 
                                                    'margin':'auto',
                                                    'margin-top':'10px',
                                                    'margin-bottom':'10px',
                                                    'color': mint, 
                                                    'fontSize': 25,
                                                    'font-weight': 'bold'}),
            dcc.RangeSlider(
                id='slider',
                min=0,
                max=100,
                step=1,
                value=[0, 100],
                marks={
                    0: {'label':'0'},
                    25: {'label':'25'},
                    50: {'label':'50'},
                    75: {'label':'75'},
                    100: {'label':'100'}
                }
            )
        ]
    
    elif tab == 'Search Engine':
        return html.Div([html.Br(),
                         dcc.Dropdown(
                             id='demo-dropdown2',
                             options= opt2,
                             value='To Kill a Mockingbird',
#                              style={"width":"1px"}
                         ), html.Br(),
                         
                         html.Div(html.Center(id='url1')),
                         
                         html.Div(
                             html.Table([
                                 html.Tr([html.Th(html.H6(' Title:',style={"color": 'black', "font-size":"16px",'padding':'1px',"margin": "1px", "margin-left": "10px"})), html.Td(id='Title')], style={"border": "2px solid gray", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Author(s):',style={"color": 'black', "font-size":"16px", 'padding':'1px',"margin": "1px","margin-left": "10px"})), html.Td(id='Author')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Average rating:',style={"color": 'black', "font-size":"16px",'padding':'1px',"margin": "1px", "margin-left": "10px"})), html.Td(id='Average_rating')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Genres:',style={"color": 'black','font-size':"16px",'padding':'1px',"margin": "1px", "margin-left": "10px"})), html.Td(id='Genres')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Number of Pages:',style={"color": 'black', "font-size":"16px",'padding':'1px',"margin": "1px", "margin-left": "10px"})), html.Td(id='Pages')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Keywords from review(s):',style={"color": 'black', "font-size":"16px",'padding':'1px',"margin": "1px", "margin-left": "10px"})), html.Td(id='Keywords')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' More information about the Author(s):',style={"color": 'black', "font-size":"16px", 'padding':'1px',"margin": "1px","margin-left": "10px"})), html.Td(id='Info')], style={"border": "2px solid grey", "font-size":"14px"}),
                                 html.Tr([html.Th(html.H6(' Description:',style={"color": 'black', "font-size":"16px", 'padding':'1px',"margin": "1px","margin-left": "10px"})), html.Td(id='Desc')], style={"border": "2px solid grey", "font-size":"14px"}),
                             ], style={"border-spacing": "10px",
                                       "border": "2px #57d2eb",
                                       'float':'right',
                                       "margin":"auto",
                                       'width':'70%',
                                       'margin-top':'10px',
                                       "background-color": "Aliceblue",
                                       "opacity": "0.8"
#                                        "width":"1100px"
                                      }))])
    
    elif tab == 'Make a Recommandation':
        return html.Div(children =[html.Br(),
                       dcc.Dropdown(
                           id='demo1-dropdown',
                           options= opt1,
                           value=''
                       ),
                       html.Br(),
                       html.Div( html.Center(id='url'),style={'width':'50%'}),
                       html.Table([
                           html.Tr([html.Td(html.H6('Top Reccomendation'),style=h6), html.Td(id='top-rec')]),
                           html.Tr([html.Td(html.H6('Author'),style=h6), html.Td(id='author')]),
                           html.Tr([html.Td(html.H6('Description'),style=h6), html.Td(id='desc')]),
                           html.Tr([html.Td(html.H6('Rating'),style=h6), html.Td(id='rating')]),
                           html.Tr([html.Td(html.H6('Length'),style=h6), html.Td(id='length')]),

                       ],style={'background-color':'#e4f2e9',
                               'opacity':'0.8'}),
            ])
    
@app.callback(
    Output('Title', 'children'),
    Output('Author', 'children'),
    Output('Average_rating', 'children'),
    Output('Genres', 'children'),
    Output('Pages', 'children'),
    Output('Keywords', 'children'),
    Output('Desc', 'children'),
    Output('Info', 'children'),
    Output('url1', 'children'),
    [dash.dependencies.Input('demo-dropdown2', 'value')])
def update_output2(value):
    x = book_engine(value)
    img = html.Img(src= x[8], style={'float':'left','width':'20%',"min-width": "300px", "height": "400px",'margin-bottom':'20px','margin':'auto'})
    return x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], img

@app.callback(
    Output('top-rec', 'children'),
    Output('author', 'children'),
    Output('desc', 'children'),
    Output('rating', 'children'),
    Output('length', 'children'),
    Output('url', 'children'),
    [dash.dependencies.Input('demo1-dropdown', 'value')])
def update_output2(value):
    x = recommend_desc(value)
    img = html.Img(src= x[6])
    l = '{} pages.'.format(str(int(x[4])))
    return x[1],  x[5],  x[2], x[3], l, img

@app.callback(
    Output("graph_attributes", "figure"),
    [Input("column_seperated", "value"),
     Input("generes", "value"),
     Input('slider', 'value')])
def display_chart(attr, genere,slider):
    df=pd.read_csv('../data/books_added_amzn.csv',error_bad_lines = False)
    if(genere != 'All'):
        df = df[df['genres'].astype(str).str.contains(genere)]
#         df = df[df['genres'].astype(str).str.contains('Fiction')]['book_rating'].to_frame()
    df[attr].to_frame()
    df.sort_values(by=[attr])
    upper_lim = int(df.shape[0]*slider[1]/100)
    lower_lim = int(df.shape[0]*slider[0]/100)
    df = df.loc[lower_lim:upper_lim]

    
    fig = px.histogram(df, x=attr,color_discrete_sequence=[sky])
    return fig

# @app.callback(
#     Output("graph", "figure"), 
#     [Input("column", "value")])
# def display_hist(attr):
#     df=pd.read_csv('../data/books_added_amzn.csv',error_bad_lines = False)
#     fig = px.histogram(df, x=attr)
#     return fig


if __name__ =='__main__':
    app.run_server(port=4050)

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

Dash is running on http://127.0.0.1:4050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4050/ (Press CTRL+C to quit)
127.0.0.1 - - [31/May/2021 22:48:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 22:48:48] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 22:48:48] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2021 22:48:48] "POST /_dash-update-component HTTP/1.1" 200 -


In [26]:
df1.iloc[df1[df1['book_title'] == 'the da vinci code'.lower()].index[0],:]['genres'].split('|')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [12]:
(df1['book_title'] == 'the da vinci code').unique()

array([False])

In [7]:
df1.head()

,Unnamed: 0,book_authors,book_desc,book_edition,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,0,J.k. Rowling | Mary Grandpré,There is a door at the end of a silent corrido...,US Edition,870,4.48,2041594,33264,Harry Potter And The Order Of The Phoenix,Manga | Sequential Art,https://images.gr-assets.com/books/1255614970l...
1,1,J.r.r. Tolkien,لجزء الثالث من ملحمة جيه أر أر تولكين الرائعة ...,Hobbit Movie Tie-in Boxed set,1728,4.59,99793,1652,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit An...,Abandoned | Baseball | Sports,https://images.gr-assets.com/books/1346072396l...
2,2,Douglas Adams,Seconds before the earth is demolished to make...,NaN,193,4.21,1155911,23919,The Hitchhiker'S Guide To The Galaxy,Baseball | Nonfiction | Sports | Sports and Games,https://images.gr-assets.com/books/1388282444l...
3,3,Shel Silverstein,"""once there was a tree...and she loved a littl...",NaN,64,4.37,789681,15694,The Giving Tree,Adult Fiction | Coming Of Age | Contemporary |...,https://images.gr-assets.com/books/1174210942l...
4,4,Dan Brown,An ingenious code hidden in the works of leona...,NaN,481,3.81,1668594,43699,The Da Vinci Code,Abandoned | Art | Contemporary | Feminism | Fi...,https://images.gr-assets.com/books/1303252999l...


In [7]:
df2.head()

,Unnamed: 0,title,authors,description,number of pages,average rating,genres,reviews' keywords,more about author(s),image_url
0,2,To Kill a Mockingbird,Harper Lee,*Amazon best selling book 2009-2019* The u...,324 pages,4.27 out of 79450 reviews,Classics | Fiction | Historical | Histor...,*amazing* *american* *best* *black* *boy* ...,*Harper Lee*: gender: male | Country: United K...,https://images.gr-assets.com/books/1361975680l...
1,32,Anne of Green Gables,L.M. Montgomery,As soon as Anne Shirley arrives at the snug wh...,320 pages,4.24 out of 16797 reviews,Classics | Fiction | Young Adult | Child...,*amazing* *bad* *better* *certain* *classi...,*L.M. Montgomery*: gender: unknown | average r...,https://images.gr-assets.com/books/1390789015l...
2,117,Tuesdays with Morrie,Mitch Albom|Saulius Dagys,"Maybe it was a grandparent, or a teacher or a ...",210 pages,4.08 out of 21367 reviews,Nonfiction | Autobiography | Memoir | Bi...,*enjoy* *everything* *final* *good* *great...,*Mitch Albom*: website: http://www.nannafoss.d...,https://images.gr-assets.com/books/1423763749l...
3,141,The Divine Comedy,Dante Alighieri|Allen Mandelbaum|Eugenio Montale,The Divine Comedy describes Dante's descent in...,798 pages,4.07 out of 3127 reviews,Classics | Poetry | Fiction | Literature,*anything* *better* *close* *different* *d...,*Dante Alighieri*: website: http://www.terri.c...,https://images.gr-assets.com/books/1390760180l...
4,142,The Compleat Works of Wllm Shkspr (abridged),Reduced Shakespeare Company|Jess Borgeson|Adam...,Revised from the rather long original complete...,137 pages,4.47 out of 127 reviews,Plays | Classics | Humor | Fiction | D...,*better* *complete* *funny* *good* *great*...,*Reduced Shakespeare Company*: gender: unknown...,https://images.gr-assets.com/books/1347579233l...


In [10]:
df1 = pd.read_csv('../book-reccomendation-engine/books_cleaned.csv')
df2 = pd.read_csv('../keyword_engine/books_authors_final.csv')
book_titles_dict1 = df1['book_title'].to_dict()
opt1 = []
for k,v in book_titles_dict1.items():
    temp_d = {}
    temp_d["label"] = v
    temp_d["value"] = v
    opt1.append(temp_d)
book_titles_dict2 = df2['title'].to_dict()
opt2 = []
for k,v in book_titles_dict2.items():
    temp_d = {}
    temp_d["label"] = v
    temp_d["value"] = v
    opt2.append(temp_d)

In [16]:
def recommend_desc(book):

    if len(df1[df1['book_title'] == book.lower()]) > 0:
        desc = list(df1[df1['book_title'] == book.lower()]['book_desc'])[0]
        print('Found match: ', book, '\n')
        match = book
    else:
        index = np.argmax([fuzz.ratio(book.lower(), i) for i in list(df1['book_title']) if type(i)== str])
        desc = df1.iloc[index,:]['book_desc']
        print('Found closest match: ', df1.iloc[index,:]['book_title'], '\n')
        match = df1.iloc[index,:]['book_title']

    all_desc = list(df1['book_desc'])
    all_genres = list(df1['genres'])
    similarity_array = np.zeros([len(all_desc),])
#     genre_input = df1.iloc[df1[df1['book_title'] == 'the da vinci code'.lower()].index[0],:]['genres'].split('|')
    genre_input=[]
    for k, i in enumerate(all_desc):
        if len(list(set(genre_input) & set(all_genres[k].split('|')))) > 2:
            if type(i)==str:
                value = get_cosine_sim(i, desc)
                if value != math.nan:
                    similarity_array[k] = get_cosine_sim(desc, i)
    similarity = similarity_array.tolist()
    similarity.remove(max(similarity))
    final_index = np.nanargmax(similarity)

#     response = requests.get(df1.iloc[final_index,:]['image_url'])

#     img = Image.open(BytesIO(response.content))

    return [match, df1.iloc[final_index,:]['book_title'], df1.iloc[final_index,:]['book_desc'],
            df1.iloc[final_index,:]['book_rating'], df1.iloc[final_index,:]['book_pages'], df1.iloc[final_index,:]['book_authors']]
x = recommend_desc('Harry Potter And The Order Of The Phoenix')
x

Found closest match:  Harry Potter And The Order Of The Phoenix 



['Harry Potter And The Order Of The Phoenix',
 'Harry Potter And The Order Of The Phoenix',
 'There is a door at the end of a silent corridor. and it’s haunting harry pottter’s dreams. why else would he be waking in the middle of the night, screaming in terror?harry has a lot on his mind for this, his fifth year at hogwarts: a defense against the dark arts teacher with a personality like poisoned honey; a big surprise on the gryffindor quidditch team; and the looming terror of the ordinary wizarding level exams. but all these things pale next to the growing threat of he-who-must-not-be-named---a threat that neither the magical government nor the authorities at hogwarts can stop.as the grasp of darkness tightens, harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice.his fate depends on them alll.(back cover)',
 4.48,
 870,
 'J.k. Rowling   | Mary Grandpré']

Found closest match:  Harry Potter And The Order Of The Phoenix 



ConnectionError: HTTPSConnectionPool(host='images.gr-assets.com', port=443): Max retries exceeded with url: /books/1255614970l/2.jpg (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fb5e0ef8450>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))